In [27]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import KFold, train_test_split
from torch.utils.data import TensorDataset, DataLoader, random_split, ConcatDataset, Subset

In [28]:
batch_size = 128
pos = pd.read_csv('/Users/jiaming/Desktop/Lab2/datas/framingham_pos.csv')
pos_unlabelled = pos.iloc[:, :-1]

neg = pd.read_csv('/Users/jiaming/Desktop/Lab2/datas/framingham_neg.csv')
neg = neg.sample(n=3000, random_state=42) # ramdomly sample the equal number of rows as pos
neg_unlabelled = neg.iloc[:, :-1]

In [29]:
datas = np.concatenate((pos_unlabelled, neg_unlabelled), axis = 0)
datas = np.expand_dims(datas, axis = -2)
labels = np.concatenate(([1] * pos.shape[0], [0] * neg.shape[0]), axis = 0)
labels = np.expand_dims(labels, -1)
datas_tensor = torch.tensor(datas, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)
dataset = TensorDataset(datas_tensor, labels_tensor)
train_size = int(0.8 * len(dataset))
train_size_1 = int(0.8 * train_size)
val_size = train_size - train_size_1
train_size = train_size_1
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
loaded_train_loader = DataLoader(train_dataset, batch_size = batch_size)
loaded_val_loader = DataLoader(val_dataset, batch_size = batch_size)
loaded_test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)

In [30]:
# save datas
torch.save({
    'train_dataset': train_dataset,  
    'val_dataset': val_dataset,
    'test_dataset': test_dataset
}, '/Users/jiaming/Desktop/Lab2/datas/saved_datasets.pth')

In [31]:
# load data from saved path
batch_size = 128
loaded_datasets_info = torch.load('/Users/jiaming/Desktop/Lab2/datas/saved_datasets.pth')
loaded_train_dataset = loaded_datasets_info['train_dataset']
loaded_val_dataset = loaded_datasets_info['val_dataset']
loaded_test_dataset = loaded_datasets_info['test_dataset']
loaded_train_loader = DataLoader(loaded_train_dataset, batch_size = batch_size) 
loaded_val_loader = DataLoader(loaded_val_dataset, batch_size = batch_size) 
loaded_test_loader = DataLoader(loaded_test_dataset, batch_size = batch_size, shuffle=False) 

In [32]:
def metrics_output(preds,labels):
    true_labels = np.array(labels)
    predicted_probs = np.array(preds)
    binary_predictions = (predicted_probs >= 0.5).astype(int)
    auc = roc_auc_score(true_labels, predicted_probs)
    conf_matrix = confusion_matrix(true_labels, binary_predictions)
    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = accuracy_score(true_labels, binary_predictions)
    f1 = f1_score(true_labels, binary_predictions)
    mcc = matthews_corrcoef(true_labels, binary_predictions)  
    return (auc, sensitivity, specificity, accuracy, f1, mcc)

In [33]:
class Net_conv(nn.Module):
    def __init__(self, input_length):
        super(Net_conv, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1)
        self.dropout1 = nn.Dropout(0.1)  
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.dropout2 = nn.Dropout(0.1) 
        flattened_size = 64 * input_length
        self.fc1 = nn.Linear(flattened_size, 64)
        self.dropout3 = nn.Dropout(0.1)  
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.dropout1(x)  
        x = self.relu(self.conv2(x))
        x = self.dropout2(x)  
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x)) 
        x = self.dropout3(x)  
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [34]:
model = Net_conv(input_length=15)  
criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.0001)
num_epochs = 50

In [35]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_dataset = ConcatDataset([loaded_train_dataset, loaded_val_dataset])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

for fold_idx, (train_index, val_index) in enumerate(kf.split(train_loader.dataset)):
    predicted_probabilities = []
    true_labels = []
    train_loader_fold = DataLoader(Subset(train_loader.dataset, train_index), batch_size=batch_size)
    val_loader_fold = DataLoader(Subset(train_loader.dataset, val_index), batch_size=batch_size)
    model = Net_conv(input_length=15)  
    
    for epoch in range(num_epochs):
        model = model.train()
        for batch_idx, (features, targets) in enumerate(train_loader_fold):     
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets) 
            loss.backward()
            optimizer.step()

    with torch.set_grad_enabled(False):
        for batch_idx, (features, targets) in enumerate(val_loader_fold):
            logits = model(features.float())
            probabilities = logits
            predicted_probabilities.extend(probabilities.tolist())
            true_labels.extend(targets.tolist())
    #torch.save(model.state_dict(), f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/CNN/CNN_{fold_idx}.pth')
    #np.save(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/CNN/y_val_pred_{fold_idx}.npy', predicted_probabilities)
    #np.save(f'D:/Jupyter_directory/Res-F5C-main/Results(Yue)/Geo/CNN/y_val_{fold_idx}.npy', true_labels)

In [36]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.4749750946403666 0.0 1.0 0.8198970840480274 0.0 0.0


In [37]:
predicted_probabilities = []  
true_labels = []  
with torch.set_grad_enabled(False): 
    for batch_idx, (features, targets) in enumerate(loaded_test_loader):
        logits = model(features.float())
        probabilities = logits
        predicted_probabilities.extend(probabilities.tolist()) 
        true_labels.extend(targets.tolist())

In [38]:
roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC = metrics_output(predicted_probabilities,true_labels)
print(roc_auc, metrics_sn, metrics_sp, metrics_ACC, metrics_F1, metrics_MCC)

0.475940307598599 0.007575757575757576 0.998324958123953 0.8189300411522634 0.01492537313432836 0.04344068035859375
